In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotion-dataset/validation.csv
/kaggle/input/emotion-dataset/training.csv
/kaggle/input/emotion-dataset/test.csv


In [8]:
import  pandas as pd 
import numpy as np
import string,time
import re
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestClassifier

In [9]:
df=pd.read_csv('/kaggle/input/emotion-dataset/training.csv')
df_val=pd.read_csv('/kaggle/input/emotion-dataset/validation.csv')

In [10]:
df.head(10)

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3
5,ive been feeling a little burdened lately wasn...,0
6,ive been taking or milligrams or times recomme...,5
7,i feel as confused about life as a teenager or...,4
8,i have been with petronas for years i feel tha...,1
9,i feel romantic too,2


preprocessing


In [11]:
df['text']=df['text'].str.lower()
df.shape


(16000, 2)

In [12]:
print(format (np.bincount(df.label)))

[4666 5362 1304 2159 1937  572]


In [13]:

def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+|pic.twitter.com\S+')
    return pattern.sub(r'',text)
df['text']=df['text'].apply(remove_url)


In [14]:

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
exclude = string.punctuation
def remove_punc(text):
    return text.translate(str.maketrans('','',exclude))
df['text']=df['text'].apply(remove_punc)


In [16]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens=[convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

In [17]:
df['text']=df['text'].apply(lambda x: convert_abbrev_in_text(x))

In [18]:
def emoji_remove(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
df['text'] = df['text'].apply(lambda x: emoji_remove(x))


In [19]:
import spacy
nlp = spacy.load('en_core_web_sm')


def preprocess_text(text):
  

    # Tokenization and lemmatization using spaCy
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.is_alpha  ]  # Lemmatization and stop word removal
    
    return ' '.join(tokens)
df['text'] = df['text'].apply(preprocess_text)

In [20]:
df1=df['text']
df.head(10)

,text,label
0,I do not feel humiliate,0
1,I can go from feel so hopeless to so damned ho...,0
2,I m grab a minute to post I feel greedy wrong,3
3,I be ever feel nostalgic about the fireplace I...,2
4,I be feel grouchy,3
5,I ve be feel a little burden lately be not sur...,0
6,I ve be take or milligram or time recommend am...,5
7,I feel as confused about life as a teenager or...,4
8,I have be with petrona for year I feel that pe...,1
9,I feel romantic too,2


# **BAG OF WORDS**

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer()

bow=cv.fit_transform(df['text'])
uniqw=cv.get_feature_names_out()#vocabulary
print(uniqw)

['aa' 'aaaaaaand' 'aaaaand' ... 'zum' 'zumba' 'zz']


In [17]:
bow_matrix=bow.toarray()

In [109]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

df['label_encoded'] = labelencoder.fit_transform(df['label'])
labels=df['label_encoded'] 
X_train1,X_test1,y_train1,y_test1=train_test_split(bow,labels,test_size=0.2,random_state=42)

In [20]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV 

param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01], 
              'kernel': ['linear']}  
  
grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3) 
grid.fit(X_train1, y_train1)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.841 total time= 1.4min
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.852 total time= 1.4min
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.850 total time= 1.4min
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.854 total time= 1.4min
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.845 total time= 1.4min
[CV 1/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.841 total time= 1.4min
[CV 2/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.852 total time= 1.4min
[CV 3/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.850 total time= 1.4min
[CV 4/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.854 total time= 1.4min
[CV 5/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.845 total time= 1.4min
[CV 1/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.841 total time= 1.4min
[CV 2/5] END ..C=0.1, gamma=0.01, kernel=linear;

GridSearchCV(estimator=SVC(probability=True),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['linear']},
             verbose=3)

In [21]:
print(grid.best_params_)

{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}


In [22]:
print(grid.best_estimator_)

SVC(C=0.1, gamma=1, kernel='linear', probability=True)


In [23]:


y_pred1=grid.predict(X_test1)
print(classification_report(y_test1,y_pred1))

              precision    recall  f1-score   support

           0       0.91      0.92      0.91       946
           1       0.83      0.93      0.88      1021
           2       0.82      0.60      0.69       296
           3       0.88      0.82      0.85       427
           4       0.82      0.85      0.83       397
           5       0.86      0.58      0.69       113

    accuracy                           0.86      3200
   macro avg       0.85      0.78      0.81      3200
weighted avg       0.86      0.86      0.85      3200



# tfidf

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 5)
                        )
dataf=df['text'].tolist()


In [56]:
tfidf.fit(dataf)



TfidfVectorizer(min_df=5, ngram_range=(1, 5), sublinear_tf=True)

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, f1_score, classification_report
from sklearn import svm

X_train_tfidfmatrix = tfidf.transform(dataf)
X_test_tfidfmatrix = tfidf.transform(dataf)


In [58]:
X_train1,X_test1,y_train1,y_test1=train_test_split(X_train_tfidfmatrix,df['label'],test_size=0.2,random_state=42)

param_grid = {'C': [ 1, 10, 100],  
              'gamma': [1, 0.1, 0.01], 
              'kernel': ['linear','rbf']}  
  
grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3) 
grid.fit(X_train1, y_train1)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END .......C=1, gamma=1, kernel=linear;, score=0.816 total time= 2.6min
[CV 2/5] END .......C=1, gamma=1, kernel=linear;, score=0.822 total time= 2.6min
[CV 3/5] END .......C=1, gamma=1, kernel=linear;, score=0.817 total time= 2.6min
[CV 4/5] END .......C=1, gamma=1, kernel=linear;, score=0.815 total time= 2.6min
[CV 5/5] END .......C=1, gamma=1, kernel=linear;, score=0.816 total time= 2.6min
[CV 1/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.730 total time= 3.7min
[CV 2/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.728 total time= 3.7min
[CV 3/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.727 total time= 3.7min
[CV 4/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.721 total time= 3.7min
[CV 5/5] END ..........C=1, gamma=1, kernel=rbf;, score=0.714 total time= 3.7min
[CV 1/5] END .....C=1, gamma=0.1, kernel=linear;, score=0.816 total time= 2.6min
[CV 2/5] END .....C=1, gamma=0.1, kernel=linear;

GridSearchCV(estimator=SVC(probability=True),
             param_grid={'C': [1, 10, 100], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['linear', 'rbf']},
             verbose=3)

In [59]:
print(grid.best_params_)

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}


In [60]:

y_pred=grid.predict(X_test1)
print(classification_report(y_test1,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.92      0.89       946
           1       0.83      0.92      0.87      1021
           2       0.79      0.58      0.67       296
           3       0.87      0.79      0.83       427
           4       0.85      0.75      0.79       397
           5       0.73      0.63      0.68       113

    accuracy                           0.84      3200
   macro avg       0.82      0.77      0.79      3200
weighted avg       0.84      0.84      0.84      3200



In [63]:
rfc = RandomForestClassifier(random_state=42)
X_train1,X_test1,y_train1,y_test1=train_test_split(X_train_tfidfmatrix,df['label'],test_size=0.2,random_state=42)

paramgrid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}
grid_search = GridSearchCV(estimator=rfc, param_grid=paramgrid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train1, y_train1)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy')

In [64]:
print(grid_search.best_params_)

{'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}


In [65]:
print(classification_report(y_test1,y_pred))

              precision    recall  f1-score   support

           0       0.87      0.92      0.89       946
           1       0.83      0.92      0.87      1021
           2       0.79      0.58      0.67       296
           3       0.87      0.79      0.83       427
           4       0.85      0.75      0.79       397
           5       0.73      0.63      0.68       113

    accuracy                           0.84      3200
   macro avg       0.82      0.77      0.79      3200
weighted avg       0.84      0.84      0.84      3200



# word2vec

In [26]:
from gensim.utils import simple_preprocess

df1=df['text']
df1 = df1.apply(simple_preprocess)

In [47]:
df1

0                               [do, not, feel, humiliate]
1        [can, go, from, feel, so, hopeless, to, so, da...
2            [grab, minute, to, post, feel, greedy, wrong]
3        [be, ever, feel, nostalgic, about, the, firepl...
4                                      [be, feel, grouchy]
                               ...                        
15995    [just, have, very, brief, time, in, the, beanb...
15996    [be, now, turn, and, feel, pathetic, that, be,...
15997                   [feel, strong, and, good, overall]
15998    [feel, like, this, be, such, rude, comment, an...
15999    [know, lot, but, feel, so, stupid, because, ca...
Name: text, Length: 16000, dtype: object

In [48]:
import gensim
import gensim.downloader as api

In [ ]:
model = api.load('word2vec-google-news-300')
vocab = model.key_to_index.keys()

In [81]:
X = df1.apply(lambda sent: model.get_mean_vector([word for word in sent if word in vocab]) )
print(X)

0        [0.028407581, 0.009945098, 0.02018758, 0.03206...
1        [0.010503616, -0.0050954977, 0.015132333, 0.04...
2        [0.01341071, 0.0122751305, -0.04660495, 0.0117...
3        [0.022345971, 0.002655724, 0.03575102, 0.04176...
4        [-0.019834952, 0.013410603, -0.007409734, 0.02...
                               ...                        
15995    [0.00036199414, 0.013361029, 0.018402299, 0.06...
15996    [-0.013023778, 0.010749258, 0.023726596, 0.048...
15997    [-0.0015173629, 0.051322863, -0.023993667, 0.0...
15998    [-0.0121134175, 0.014537073, 0.0134658115, 0.0...
15999    [0.017762095, 0.003723685, 0.006926316, 0.0425...
Name: text, Length: 16000, dtype: object


In [88]:
X1 = np.vstack(X)
Y=np.array(df['label'])
print(X)

[[ 0.02840758  0.0099451   0.02018758 ... -0.0509265   0.01485861
  -0.03793244]
 [ 0.01050362 -0.0050955   0.01513233 ... -0.02066479  0.04158253
  -0.02196515]
 [ 0.01341071  0.01227513 -0.04660495 ... -0.05027774 -0.01293708
  -0.00779253]
 ...
 [-0.00151736  0.05132286 -0.02399367 ... -0.02990308  0.0335833
  -0.04242011]
 [-0.01211342  0.01453707  0.01346581 ... -0.02455021  0.03368159
  -0.0403529 ]
 [ 0.01776209  0.00372368  0.00692632 ... -0.03305596  0.04279935
  -0.02570457]]


In [83]:
def create_features(df):
    
 
    X = df['text'].apply(lambda sent: model.get_mean_vector([word for word in sent if word in vocab]))
    X = np.vstack(X)
       
    return X

In [90]:
X_val=create_features(df)
y_val = np.array(Y)

In [105]:
X_train1,X_test1,y_train1,y_test1=train_test_split(X,labels,test_size=0.2,random_state=42)

In [101]:
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01], 
              'kernel': ['linear']}  
  
grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3) 
grid.fit(X_train1, y_train1)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.448 total time= 1.5min
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.436 total time= 1.4min
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.441 total time= 1.5min
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.442 total time= 1.5min
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.446 total time= 1.4min
[CV 1/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.448 total time= 1.5min
[CV 2/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.436 total time= 1.5min
[CV 3/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.441 total time= 1.5min
[CV 4/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.442 total time= 1.4min
[CV 5/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.446 total time= 1.4min
[CV 1/5] END ..C=0.1, gamma=0.01, kernel=linear;, score=0.448 total time= 1.4min
[CV 2/5] END ..C=0.1, gamma=0.01, kernel=linear;

GridSearchCV(estimator=SVC(probability=True),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['linear']},
             verbose=3)

In [102]:
print(grid.best_estimator_)

SVC(C=100, gamma=1, kernel='linear', probability=True)


In [106]:
y_pred1=grid.predict(X_test1)
print(classification_report(y_test1,y_pred1))

              precision    recall  f1-score   support

           0       0.67      0.75      0.71       946
           1       0.67      0.83      0.74      1021
           2       0.63      0.33      0.44       296
           3       0.68      0.54      0.60       427
           4       0.72      0.58      0.64       397
           5       0.66      0.42      0.52       113

    accuracy                           0.67      3200
   macro avg       0.67      0.58      0.61      3200
weighted avg       0.67      0.67      0.66      3200

